## Необходимые пакеты:

In [1]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import chromedriver_binary 
import urllib
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import lxml
import requests
import base64
import json
import pandas as pd
import numpy as np
from secrets import *
import urllib.request
from fake_useragent import UserAgent
from bs4 import BeautifulSoup 
from selenium.webdriver.common.action_chains  import ActionChains
import random

#####  Подготовка к работе с API, авторизируемся на сайте и изменяем кодировку для получения токена доступа.

In [2]:
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}


message = f"{'3e661d7345d04909b518ec04ec996fd2'}:{'c3ce130028654a779d6404a619845bd6'}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')

headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

r = requests.post(url, headers=headers, data=data)

token = r.json()['access_token']

In [3]:
token

'BQC-SszN0XCm0FbD2x9cHwwQ82T8riwlZWx1QhJj1ak7w1W9c2MCvG1-q4kuFwDIDrJ3C1uWFoXY5rcPEWc'

Успешно получен токен.

###### Функция получения сведений о треках, находящихся в одном плейлисте

In [4]:
def get_playlist(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    page = playlist_req_to_list (page)    
    return page


###### Функция по вытягиванию параметров песни

In [5]:
def song_paramets(song_id):
    
    song_url = f'https://api.spotify.com/v1/audio-features/{song_id}'
    headers = {
        "Authorization": "Bearer " + token
    }

    page = requests.get(url=song_url, headers=headers)
    page =  page.json()
    return page

###### Функция распредения инфомации из json-формата в DataFrame-таблицу


In [6]:
def playlist_req_to_list(file):

    song_name = [None for i in range(50)]
    song_id = [None for i in range(50)]
    duration = [None for i in range(50)]
    artist_name = [None for i in range(50)]
    artist_id = [None for i in range(50)]
    album_name = [None for i in range(50)]
    album_id = [None for i in range(50)]
    release_date = [None for i in range(50)]
    local_position = [None for i in range(50)]
    world_position = [None for i in range(50)]
    danceability = [None for i in range(50)]
    energy = [None for i in range(50)]
    key = [None for i in range(50)]
    loudness = [None for i in range(50)]
    mode = [None for i in range(50)]
    speechiness = [None for i in range(50)]
    acousticness = [None for i in range(50)]
    instrumentalness = [None for i in range(50)]
    liveness = [None for i in range(50)]
    valence = [None for i in range(50)]
    tempo = [None for i in range(50)]
      
    i = 0
    
    for item in file['items']:        
        song_name[i] = item['track']['name']
        song_id[i] = item['track']['id']
        duration[i] = item['track']['duration_ms']
        local_position[i] = item['track']['popularity']       
        artist_name[i] = item['track']['album']['artists'][0]['name']
        artist_id[i] = item['track']['album']['artists'][0]['id']
        album_name[i] = item['track']['album']['name']
        album_id[i] = item['track']['album']['id']
        release_date[i] = item['track']['album']['release_date']
        
        parametrs = song_paramets(song_id[i])
        
        danceability[i] = parametrs['danceability']
        energy[i] = parametrs['energy']
        key[i] = parametrs['key']
        loudness[i] = parametrs['loudness']
        mode[i] = parametrs['mode']
        speechiness[i] = parametrs['speechiness']
        acousticness[i] = parametrs['acousticness']
        instrumentalness[i] = parametrs['instrumentalness']
        liveness[i] = parametrs['liveness']
        valence[i] = parametrs['valence']
        tempo[i] = parametrs['tempo']
        
        i += 1
        
    data_dict = {'song_name' : song_name,
                 'song_id' : song_id,
                 'duration_ms' : duration,
                 'local_position' : local_position,
                 'artist_name' : artist_name,
                 'artist_id' : artist_id,
                 'album_name' : album_name,
                 'album_id' : album_id,
                 'release_date' : release_date,
                 'danceability': danceability,
                 'energy': energy,
                 'key': key,
                 'loudness': loudness,
                 'mode': mode,
                 'speechiness': speechiness,
                 'acousticness': acousticness,
                 'instrumentalness': instrumentalness,
                 'liveness': liveness,
                 'valence': valence,
                 'tempo': tempo
                }
    data_set = pd.DataFrame(data_dict)
    
    return data_set

##### Функция по получению основной инфы о плейлисте

In [7]:
def playlist_info(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    text = page['description']
    number = page['followers']['total']
    country = text[text.find('now -')+6:-1]
    info = [playlist_id,country,number]
    return info

##### Функция по отбору локальной инфы для песен

In [38]:
def get_MainInfo(playlist, playlist_id):
    #создаем часть таблицы (место в топе в качестве индекса, имена и id песен)
    MainInfo = playlist[['song_name', 'song_id']]
    
    #парсим код страницы альбома с испльзованием селениума
    site = "https://open.spotify.com/playlist/" + playlist_id
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(site)
    time.sleep(2)
    response = driver.page_source
    driver.close()
    tree = BeautifulSoup(response, 'html.parser')
    
    #ищем все куски кода, где написаны просмотры
    a = tree.find_all('div', class_ = 'd47b790d001ed769adcd9ddfc0e83acc-scss f3fc214b257ae2f1d43d4c594a94497f-scss')
    
    #из них получаем строки с прослушиваниями и обрабатываем их
    res = []
    b = list(a)
    for i in b:
        views = str(i)[-16:-6].replace('\xa0', '')
        res.append(views)
    #res - список прослушиваний от топа ко дну (неформатированный)
    #вытащим из элементов res только числа
    views = []
    for i in res:
        ddd = [s for s in i.split('>') if s.isdigit()]
        views.append(int(ddd[0]))
    #views - просмотры по порядку для каждой песни из топа
    #добавляем колонки с просмотрами, местом в топе и страной к существующей таблице
    MainInfo.insert(2, "views", views)
    MainInfo.insert(0, "place", range(1,51))
    MainInfo.insert(0, "country", playlist_info(playlist_id)[0])
    return MainInfo

## Находим все айдишники альбомов 
#### Полученное ids-список айдишников

In [8]:
site = "https://open.spotify.com/genre/charts-regional-weekly"

#chrome_options = Options() работает у Дениса
#chrome_options.add_argument("--headless")   настройка на всякий случай

#настраиваем селениум
#driver = webdriver.Chrome('D:\chromedriver.exe') работает у Дениса
driver = webdriver.Chrome('/Users/alexandr/Downloads/chromedriver') #работает у Сани
#открываем сайт, двигаем мышку и кликаем, чтобы потом заработала прокрутка
driver.get(site)
driver.maximize_window()
time.sleep(3)
ActionChains(driver).move_by_offset(2000, 900).click().perform() #у дениса 1500 350
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) 
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) #моя добавка
time.sleep(3)


#получаем код страницы и делаем суп
response = driver.page_source
driver.close()
tree = BeautifulSoup(response, 'html.parser')

In [9]:
#ищем все ссылки на странице
a = tree.find_all('a', class_ = 'f7ebc3d96230ee12a84a9b0b4b81bb8f-scss')

#из блоков с ссылками достаем айдишники
ids = []
for i in a:
    id = i.get('href')
    ids.append(id[10:])
ids

['37i9dQZEVXbNG2KDcFcKOF',
 '37i9dQZEVXbK4fwx2r07XW',
 '37i9dQZEVXbM1EaZ0igDlz',
 '37i9dQZEVXbKPTKrnFPD0G',
 '37i9dQZEVXbND4ZYa46PaA',
 '37i9dQZEVXbKzoK95AbRy9',
 '37i9dQZEVXbMwmF30ppw50',
 '37i9dQZEVXbMYsavqzfk6k',
 '37i9dQZEVXbKZyn1mKjmIl',
 '37i9dQZEVXbJHSzlHx2ZJU',
 '37i9dQZEVXbK8BKKMArIyl',
 '37i9dQZEVXbMdvweCgpBAe',
 '37i9dQZEVXbMw2iUtFR5Eq',
 '37i9dQZEVXbMPoK06pe7d6',
 '37i9dQZEVXbMy2EcFg5F9m',
 '37i9dQZEVXbJ5J1TrbkAF9',
 '37i9dQZEVXbMWDif5SCBJq',
 '37i9dQZEVXbIZK8aUquyx8',
 '37i9dQZEVXbJIvhIOxXxdp',
 '37i9dQZEVXbJwoKy8qKpHG',
 '37i9dQZEVXbJUPkgaWZcWG',
 '37i9dQZEVXbMda2apknTqH',
 '37i9dQZEVXbL1Fl8vdBUba',
 '37i9dQZEVXbKcS4rq3mEhp',
 '37i9dQZEVXbNM8vS9cIqAG',
 '37i9dQZEVXbKUoIkUXteF6',
 '37i9dQZEVXbK4BFAukDzj3',
 '37i9dQZEVXbIWlLQoMVEFp',
 '37i9dQZEVXbLWYFZ5CkSvr',
 '37i9dQZEVXbIZQf3WEYSut',
 '37i9dQZEVXbNSiWnkYnziz',
 '37i9dQZEVXbMGcjiWgg253',
 '37i9dQZEVXbMZ5PAcNTDXd',
 '37i9dQZEVXbJZGli0rRP3r',
 '37i9dQZEVXbNALwC1jxb5m',
 '37i9dQZEVXbMeCoUmQDLUW',
 '37i9dQZEVXbO839WGRmpu1',
 

##### Функция по поиску прослушиваний для каждого плейлиста


In [10]:
def viewslist(playlist_id):
    site = f'https://open.spotify.com/playlist/{playlist_id}'
    driver = webdriver.Chrome('/Users/alexandr/Downloads/chromedriver')
    driver.get(site)
    time.sleep(2)
    response = driver.page_source
    driver.close()
    tree = BeautifulSoup(response, 'html.parser')
    a = tree.find_all('div', class_ = 'd47b790d001ed769adcd9ddfc0e83acc-scss f3fc214b257ae2f1d43d4c594a94497f-scss')
    res = []
    b = list(a)
    for i in b:
        views = str(i)[-16:-6].replace('\xa0', '')
        res.append(views)
    views = []
    for i in res:
        ddd = [s for s in i.split('>') if s.isdigit()]
        views.append(int(ddd[0]))
    return views

# Cобираем датасэты

## Структура данных:
#### Маленький датасетик вида (айди листа, страна, кол-во лайков)
#### Датасет побольбше (данные о каждой песне)
#### Датасет совсем большой че с ним (данные о позиции и прослушиваниях каждой песни в каждом топе)

#### Маленький датасетик


In [11]:
countries = pd.DataFrame(playlist_info(item) for item in ids)
countries.columns= ['id', 'country', 'followers'] 
countries = countries.set_index('country')


### Датасет побольбше (данные о каждой песне)


In [26]:
allsongs=pd.DataFrame()
for item in tqdm(ids):
    allsongs=pd.concat([allsongs,get_playlist(item)],sort=False, axis=0)

100%|██████████| 53/53 [05:36<00:00,  6.35s/it]


In [37]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
allsongs = allsongs.drop_duplicates(subset=['song_id'])
allsongs = allsongs.set_index('song_id')
allsongs

,song_name,duration_ms,local_position,artist_name,artist_id,album_name,album_id,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
song_id,,,,,,,,,,,,,,,,,,,
4ZtFanR9U6ndgddUvNcjcG,good 4 u,178146,93,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,SOUR,6s84u2TUpR3wdUv4NgKA2j,2021-05-21,0.563,0.6640,9,-5.044,1,0.1540,0.335000,0.000000,0.0849,0.6880,166.928
3VqeTFIvhxu3DIe4eZVzGq,Butter,164441,98,BTS,3Nrfpe0tUJi4K4DXYWgMUX,Butter,2BDhPi2XCYujYxU6VM0QaD,2021-05-21,0.759,0.4590,8,-5.187,1,0.0948,0.003230,0.000000,0.0906,0.6950,109.997
6HU7h9RYOaPRFeh0R3UeAr,deja vu,215506,89,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,SOUR,6s84u2TUpR3wdUv4NgKA2j,2021-05-21,0.442,0.6120,2,-7.222,1,0.1120,0.584000,0.000006,0.3700,0.1780,180.917
67BtfxlNbhBmCDR2L2l8qd,MONTERO (Call Me By Your Name),137875,100,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,MONTERO (Call Me By Your Name),2Hjcfw8zHN4dJDZJGOzLd6,2021-03-31,0.610,0.5080,8,-6.682,0,0.1520,0.297000,0.000000,0.3840,0.7580,178.818
748mdHapucXQri7IAO8yFK,Kiss Me More (feat. SZA),208866,98,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Kiss Me More (feat. SZA),1OnzqJTL9bwe4kvaLxRYxt,2021-04-09,0.762,0.7010,8,-3.541,1,0.0286,0.235000,0.000158,0.1230,0.7420,110.968
5wANPM4fQCJwkGd4rN57mH,drivers license,242013,88,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,SOUR,6s84u2TUpR3wdUv4NgKA2j,2021-05-21,0.561,0.4310,10,-8.810,1,0.0578,0.768000,0.000014,0.1060,0.1370,143.875
5CZ40GBx1sQ9agT82CLQCT,traitor,229226,91,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,SOUR,6s84u2TUpR3wdUv4NgKA2j,2021-05-21,0.380,0.3390,3,-7.885,1,0.0338,0.691000,0.000000,0.1200,0.0849,100.607
4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),198081,99,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,Justice,5dGWwsZ9iB2Xc3UKR0gif2,2021-03-19,0.677,0.6960,0,-6.181,1,0.1190,0.321000,0.000000,0.4200,0.4640,90.030
6SRsiMl7w1USE4mFqrOhHC,brutal,143746,90,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,SOUR,6s84u2TUpR3wdUv4NgKA2j,2021-05-21,0.672,0.5260,4,-6.446,1,0.0599,0.031100,0.000001,0.2800,0.2800,123.999


## Датасет совсем большой че с ним (данные о позиции и прослушиваниях каждой песни в каждом топе)